In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
model = m.pretrained_pytorch_vit_digits_model()

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'trainset4'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'trainset1'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 2000 Val size: 1000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[-0.5706,  1.2041,  0.1345, -0.1973,  0.1907,  0.2617,  0.5288,  0.1813,
          0.1364, -0.4564, -0.6791,  0.0222,  0.2097, -0.6347, -0.8362,  0.0197,
         -0.1037, -1.0422, -0.6976,  0.7924,  0.8553, -0.1847, -0.5988, -0.0505,
         -0.6408, -0.0518, -0.1626, -0.3347,  0.5116,  0.5483, -0.4732, -0.4154,
         -0.9557,  0.3977, -0.4254,  0.0924, -0.5521, -0.4383, -0.2078, -0.7467,
         -0.3036,  0.2506,  0.1612,  0.3425, -0.7730]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune'), digits_model)

Epoch 10 done, train loss: 0.0010 val acc: 0.9010
Epoch 20 done, train loss: 0.0003 val acc: 0.9010
